In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
import os


In [27]:
40*24*12


11520

## score functions

In [ ]:
def score_GVP(dy):
    ceil = 2.
    gvp = (1.0 + (dy/5.0)**2)**0.5 - 1.0
    res = np.clip(gvp, 0., ceil) # if gvp < ceil {gvp} else {ceil};
    return 100.*res


In [ ]:
data = "CGM_p77_24h"
diff = "CGMDIFF"+data[3:]

df = pd.read_csv("data/%s.csv"%data, header=None, names=["ts","CGM"])
df["TIR"] = df["CGM"].apply(lambda x: 100.0 if ((x<180) & (x>70)) else 0.0)
df["H70"] = df["CGM"].apply(lambda x: 100.0 if (x<70) else 0.0)
df["H54"] = df["CGM"].apply(lambda x: 100.0 if (x<54) else 0.0)
df["LOG"] = df["CGM"].apply(lambda x: 200.*(np.log(np.clip(x,20,400))/np.log(20)-1))
df["GVP"] = df["CGM"].diff(1).apply()
df["ADV"] = df["CGM"].diff(1).apply(np.abs)
print(df.iloc[:,1:].mean())
df.head()


## rolling averages

In [ ]:
win = 8
dfr = df.rolling(8).mean().iloc[win-1:]
dfr.reset_index(drop=True, inplace=True)
dfr


In [ ]:
#!ls data/*/*rolling*.csv

In [ ]:
allfiles = glob("data/*/%s*rolling*.csv"%data)

dtr_281 = {}
dtr_280 = {}

for filepath in allfiles:
    _, keys, filename = filepath.split("/")
    name, _ = filename.split(".")
    tmp = name.split("_")
    tag = keys+"_"+"_".join(tmp[3:])
    if "GVP" in tag or "ADV" in tag:
        dtr_280[tag] = pd.read_csv(filepath, header=None).values[:,0]
    else:
        dtr_281[tag] = pd.read_csv(filepath, header=None).values[:,0]
        


In [ ]:
idx = "CGM"
dtr = dtr_281
dsr = pd.DataFrame(dtr, index=None) #index=dfr.index
print(dsr.shape,dfr.shape)
cols = [col for col in dsr.columns if "TIR" not in col and "H70" not in col and "H54" not in col and "GVP" not in col and "ADV" not in col and "LOG" not in col]
print(cols)
plt.plot(dfr[idx])
plt.plot(dsr[cols], ".")
plt.show()
#(dsh - dfh[[idx]].values).describe().T, pd.concat([dsh,dfh[idx]], axis =1)

tmp = (dsr[cols] - dfr[[idx]].values).describe().T[["mean","std","25%","75%"]]
tmp["rmse"] = (tmp["std"]**2+tmp["mean"]**2)**0.5
tmp["rel"] = tmp["rmse"]/dfr[idx].mean()
tmp.sort_values("rel")


In [ ]:
idx = "TIR"
dtr = dtr_281
dsr = pd.DataFrame(dtr, index=None)
cols = [col for col in dsr.columns if idx  in col]
plt.plot(dfr[idx])
plt.plot(dsr[cols], ".")
plt.show()
#(dsh - dfh[[idx]].values).describe().T, pd.concat([dsh,dfh[idx]], axis =1)

tmp = (dsr[cols] - dfr[[idx]].values).describe().T[["mean","std","25%","75%"]]
tmp["rmse"] = (tmp["std"]**2+tmp["mean"]**2)**0.5
tmp["rel"] = tmp["rmse"]/dfr[idx].mean()
tmp.sort_values("rel")


In [ ]:
idx = "GVP"
dtr = dtr_280
dsr = pd.DataFrame(dtr, index=None)
cols = [col for col in dsr.columns if idx  in col]
plt.plot(dfr[idx])
plt.plot(dsr[cols], ".")
plt.show()
#(dsh - dfh[[idx]].values).describe().T, pd.concat([dsh,dfh[idx]], axis =1)

tmp = (dsr[cols] - dfr[[idx]].values[-len(dsr):]).describe().T[["mean","std","25%","75%"]]
tmp["rmse"] = (tmp["std"]**2+tmp["mean"]**2)**0.5
tmp["rel"] = tmp["rmse"]/dfr[idx].mean()
tmp.sort_values("rel")


In [ ]:
idx = "LOG"
dtr = dtr_281
dsr = pd.DataFrame(dtr, index=None)
cols = [col for col in dsr.columns if idx  in col]
plt.plot(dfr[idx])
plt.plot(dsr[cols], ".")
plt.show()
#(dsh - dfh[[idx]].values).describe().T, pd.concat([dsh,dfh[idx]], axis =1)

tmp = (dsr[cols] - dfr[[idx]].values).describe().T[["mean","std","25%","75%"]]
tmp["rmse"] = (tmp["std"]**2+tmp["mean"]**2)**0.5
tmp["rel"] = tmp["rmse"]/dfr[idx].mean()
tmp.sort_values("rel")


In [ ]:
1+3

### cost of rolling

time cost of one bootstrap

In [7]:
!tail scores_stats.txt

80_1024_1_6_4 CGM_p77_40D_2_3_400 H70 25 3595
80_1024_1_6_4 CGM_p77_40D_2_3_400 TIR 2606 3689
80_1024_1_6_4 CGM_p77_40D_2_3_400 H70 26 3578
80_1024_1_6_4 CGM_p77_40D_2_3_400 TIR 2607 3710
80_1024_1_6_4 CGM_p77_40D_2_3_400 H70 27 3670
80_1024_1_6_4 CGM_p77_40D_2_3_400 TIR 2608 3614
80_1024_1_6_4 CGM_p77_40D_2_3_400 H70 28 3754
80_1024_1_6_4 CGM_p77_40D_2_3_400 TIR 2609 3613
80_1024_1_6_4 CGM_p77_40D_2_3_400 H70 29 3600
80_1024_1_6_4 CGM_p77_40D_2_3_400 TIR 2610 3673


time cost for rolling every 5 min

In [25]:
2**10/(12*24)

3.5555555555555554

In [17]:
n = 10
m = n*4+n-1+1+2+1
n, m, 3.7*m/60

(10, 53, 3.2683333333333335)

combined cost of rolling over hourly averages (with amortisation)

In [39]:
(2**7+2**5+2**3)/24, (2**7)/24

(7.0, 5.333333333333333)

In [23]:
n = 7
m = (n*4+n-1+1+2+1) + 5*2*12
t = 3.7*m/60
n, m, m/12, t, t/12

(7, 158, 13.166666666666666, 9.743333333333334, 0.8119444444444445)